In [1]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

Depending on the model life cycle of your data science team, you would like to experiment and track training Pipeline runs and its associated parameters. Then, you would to compare runs of these Pipelines to each others in order to figure out which is the best configuration generates the model you will register in the Vertex AI Model Registry.

### Dataset

The dataset used for this tutorial is the [Iris dataset](https://www.tensorflow.org/datasets/catalog/iris) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). This dataset does not require any feature engineering. The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of Iris flower species from a class of three species: setosa, virginica, or versicolor.


### Objective

In this notebook, you will learn how to use Vertex AI Experiments to 

*   Log Pipeline Job
*   Compare different Pipeline Jobs

The steps covered include:

* Formalize a training component
* Build a training pipeline
* Run several Pipeline jobs and log their results
* Compare different Pipeline jobs


### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as Vertex AI SDK and KFP SDK. Use the latest major GA version of each package.

In [2]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip install {USER_FLAG} --upgrade google-cloud-aiplatform \
                                    google-cloud-storage \
                                    google-auth \
                                    kfp -q

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [3]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  vertex-ai-dev


Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [3]:
REGION = "us-central1"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.


In [4]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [5]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [6]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [7]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://vertex-ai-devaip-exxfumy6/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [8]:
! gsutil ls -al $BUCKET_URI

#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [9]:
SERVICE_ACCOUNT = "931647533046-compute@developer.gserviceaccount.com"  # @param {type:"string"}

In [10]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [11]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Download training dataset


In [12]:
DATASET_URI = "gs://cloud-samples-data/ai-platform/iris"

In [13]:
!gsutil cp -r $DATASET_URI $BUCKET_URI

Copying gs://cloud-samples-data/ai-platform/iris/classification/evaluate.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/classification/train.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_data.csv [Content-Type=application/octet-stream]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_predict.csv [Content-Type=text/csv]...
\ [4 files][  5.4 KiB/  5.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cloud-samples-data/ai-platform/iris/iris_target.csv [Content-Type=application/octet-stream]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_test.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_training.cs

### Import libraries and define constants

In [14]:
import logging
# General
import os
import time

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

import kfp.v2.compiler as compiler
# Pipeline Experiments
import kfp.v2.dsl as dsl
# Vertex AI
from google.cloud import aiplatform as vertex_ai
from kfp.v2.dsl import Metrics, Model, Output, component

In [15]:
# Experiments
TASK = "classification"
MODEL_TYPE = "xgboost"
EXPERIMENT_NAME = f"{PROJECT_ID}-{TASK}-{MODEL_TYPE}-{UUID}"

# Pipeline
PIPELINE_URI = f"{BUCKET_URI}/pipelines"
TRAIN_URI = f"{BUCKET_URI}/iris/iris_data.csv"
LABEL_URI = f"{BUCKET_URI}/iris/iris_target.csv"
MODEL_URI = f"{BUCKET_URI}/model"

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [16]:
vertex_ai.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### Set pre-built containers

Set the pre-built Docker container image for training and prediction.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [17]:
TRAIN_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="xgboost", framework_version="1.1", accelerator="cpu"
)

## Formalize your training as pipeline component


Before you start running your pipeline experiments, you have to formalize your training as pipeline component.

To do that, you will use the `kfp.v2.dsl.component` decorator to convert your training task into a pipeline component. 

In [18]:
@component(
    packages_to_install=[
        "numpy==1.18.5",
        "pandas==1.0.4",
        "scikit-learn==0.23.1",
        "xgboost==1.1.1",
    ]
)
def custom_trainer(
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    boost_rounds: int,
    model_uri: str,
    metrics: Output[Metrics],
    model_metadata: Output[Model],
):

    # import libraries
    import logging
    import uuid
    from pathlib import Path as path

    import pandas as pd
    import xgboost as xgb
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split

    # variables
    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    train_path = train_uri.replace(gs_prefix, gcsfuse_prefix)
    label_path = label_uri.replace(gs_prefix, gcsfuse_prefix)
    model_path = model_uri.replace(gs_prefix, gcsfuse_prefix)

    def get_logger():
        """
        Get the logger
        """
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(
            logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        )
        logger.addHandler(handler)
        return logger

    def get_data(
        train_path: str, label_path: str
    ) -> (xgb.DMatrix, pd.DataFrame, pd.DataFrame):
        """
        Get the data
        Args:
            train_path: the path of the train data
            label_path: the path of the label data
        Returns:
            the train data and the label data
        """
        # Load data into pandas, then use `.values` to get NumPy arrays
        data = pd.read_csv(train_path).values
        labels = pd.read_csv(label_path).values

        # Convert one-column 2D array into 1D array for use with XGBoost
        labels = labels.reshape((labels.size,))
        train_data, test_data, train_labels, test_labels = train_test_split(
            data, labels, test_size=0.2, random_state=7
        )

        # Load data into DMatrix object
        dtrain = xgb.DMatrix(train_data, label=train_labels)
        return dtrain, test_data, test_labels

    def train_model(max_depth: int, eta: int, boost_rounds, dtrain: xgb.DMatrix):
        """
        Train the model
        Args:
            max_depth: the max depth of the model
            eta: the eta of the model
            boost_rounds: the boost rounds of the model
            dtrain: the train data
        Returns:
            the trained model
        """
        # Train XGBoost model
        param = {"max_depth": max_depth, "eta": eta}
        model = xgb.train(param, dtrain, num_boost_round=boost_rounds)
        return model

    def evaluate_model(model, test_data, test_labels):
        """
        Evaluate the model
        Args:
            model: the trained model
            test_data: the test data
            test_labels: the test labels
        Returns:
            the accuracy of the model
        """
        dtest = xgb.DMatrix(test_data)
        pred = model.predict(dtest)
        predictions = [round(value) for value in pred]
        # Evaluate predictions
        accuracy = accuracy_score(test_labels, predictions)
        return accuracy

    def save_model(model, model_path):
        """
        Save the model
        Args:
            model: the trained model
            model_path: the path of the model
        """
        model_id = str(uuid.uuid1())
        model_path = f"{model_path}/{model_id}/model.bst"
        path(model_path).parent.mkdir(parents=True, exist_ok=True)
        model.save_model(model_path)

    # Main ----------------------------------------------

    dtrain, test_data, test_labels = get_data(train_path, label_path)
    model = train_model(max_depth, learning_rate, boost_rounds, dtrain)
    accuracy = evaluate_model(model, test_data, test_labels)
    save_model(model, model_path)

    # Metadata ------------------------------------------
    metrics.log_metric("accurancy", accuracy)
    model_metadata.uri = model_uri

## Build a pipeline

### Define your workflow using Kubeflow Pipelines DSL package

In [19]:
@dsl.pipeline(name="custom-training-pipeline")
def pipeline(
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    boost_rounds: int,
    model_uri: str,
):

    custom_trainer(
        train_uri, label_uri, max_depth, learning_rate, boost_rounds, model_uri
    )

### Compile your pipeline into a JSON file

In [20]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline.json")

/usr/local/lib/python3.9/site-packages/kfp/v2/compiler/compiler.py:1292: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


## Submit and track pipeline runs

### Submit Pipeline runs

Now that you have the pipeline, you define its training configuration depending on the defined parameters. Below you have an example and how you can submit several pipeline runs. 

In [21]:
runs = [
    {"max_depth": 4, "learning_rate": 0.2, "boost_rounds": 10},
    {"max_depth": 5, "learning_rate": 0.3, "boost_rounds": 20},
    {"max_depth": 3, "learning_rate": 0.1, "boost_rounds": 30},
    {"max_depth": 6, "learning_rate": 0.5, "boost_rounds": 40},
    {"max_depth": 5, "learning_rate": 0.4, "boost_rounds": 30},
]

In [22]:
for i, run in enumerate(runs):

    job = vertex_ai.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path="pipeline.json",
        pipeline_root=PIPELINE_URI,
        parameter_values={
            "train_uri": TRAIN_URI,
            "label_uri": LABEL_URI,
            "model_uri": MODEL_URI,
            **run,
        },
    )
    job.submit(experiment=EXPERIMENT_NAME)

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133409


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133409


To use this PipelineJob in another session:


...

INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133420')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20220809133420?project=931647533046


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20220809133420?project=931647533046


Associating projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133420 to Experiment: vertex-ai-dev-classification-xgboost-exxfumy6


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133420 to Experiment: vertex-ai-dev-classification-xgboost-exxfumy6


### Check Pipeline run states

Vertex AI SDK provides you `get_experiment_df` method to monitor the status of pipeline runs. You can use it either to return parameters and metrics of the Pipeline Runs in the Vertex AI Experiment or in combination with `get` method of `PipelineJob` to return the pipeline job in Vertex AI Pipeline.


In [23]:
# see state of all pipelineJob
vertex_ai.get_experiment_df(EXPERIMENT_NAME)

,experiment_name,run_name,run_type,state,param.train_uri,param.label_uri,param.max_depth,param.learning_rate,param.boost_rounds,param.model_uri
0,vertex-ai-dev-classification-xgboost-exxfumy6,custom-training-pipeline-20220809133420,system.PipelineRun,RUNNING,gs://vertex-ai-devaip-exxfumy6/iris/iris_data.csv,gs://vertex-ai-devaip-exxfumy6/iris/iris_targe...,5,0.4,30,gs://vertex-ai-devaip-exxfumy6/model
1,vertex-ai-dev-classification-xgboost-exxfumy6,custom-training-pipeline-20220809133418,system.PipelineRun,RUNNING,gs://vertex-ai-devaip-exxfumy6/iris/iris_data.csv,gs://vertex-ai-devaip-exxfumy6/iris/iris_targe...,6,0.5,40,gs://vertex-ai-devaip-exxfumy6/model
2,vertex-ai-dev-classification-xgboost-exxfumy6,custom-training-pipeline-20220809133415,system.PipelineRun,RUNNING,gs://vertex-ai-devaip-exxfumy6/iris/iris_data.csv,gs://vertex-ai-devaip-exxfumy6/iris/iris_targe...,3,0.1,30,gs://vertex-ai-devaip-exxfumy6/model
3,vertex-ai-dev-classification-xgboost-exxfumy6,custom-training-pipeline-20220809133413,system.PipelineRun,RUNNING,gs://vertex-ai-devaip-exxfumy6/iris/iris_data.csv,gs://vertex-ai-devaip-exxfumy6/iris/iris_targe...,5,0.3,20,gs://vertex-ai-devaip-exxfumy6/model
4,vertex-ai-dev-classification-xgboost-exxfumy6,custom-training-pipeline-20220809133409,system.PipelineRun,RUNNING,gs://vertex-ai-devaip-exxfumy6/iris/iris_data.csv,gs://vertex-ai-devaip-exxfumy6/iris/iris_targe...,4,0.2,10,gs://vertex-ai-devaip-exxfumy6/model


In [24]:
while True:
    pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
    if all(
        pipeline_state != "COMPLETE" for pipeline_state in pipeline_experiments_df.state
    ):
        print("Pipeline runs are still running...")
        if any(
            pipeline_state == "FAILED"
            for pipeline_state in pipeline_experiments_df.state
        ):
            print("At least one Pipeline run failed")
            break
    else:
        print("Pipeline experiment runs have completed")
        break
    time.sleep(60)

Pipeline runs are still running...
Pipeline runs are still running...
Pipeline experiment runs have completed


In [25]:
# Get the PipelineJob resource using the experiment run name
pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
for i in range(5):
    job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[i])
    print(job.resource_name)
    print(job._dashboard_uri())
    

projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133420
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20220809133420?project=931647533046
projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133418
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20220809133418?project=931647533046
projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133415
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20220809133415?project=931647533046
projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133413
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20220809133413?project=931647533046
projects/931647533046/locations/us-central1/pipe

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial

In [26]:
# Delete the pipeline
# Get the PipelineJob resource using the experiment run name
pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
for i in range(5):
    job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[i])
    print(job.resource_name)
    print(job._dashboard_uri())
    job.delete()

projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133420
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20220809133420?project=931647533046
Deleting PipelineJob : projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133420


INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133420


Delete PipelineJob  backing LRO: projects/931647533046/locations/us-central1/operations/4973821511749074944


INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/931647533046/locations/us-central1/operations/4973821511749074944


PipelineJob deleted. . Resource name: projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133420


...

INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133409


Delete PipelineJob  backing LRO: projects/931647533046/locations/us-central1/operations/8676906305354465280


INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/931647533046/locations/us-central1/operations/8676906305354465280


PipelineJob deleted. . Resource name: projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133409


INFO:google.cloud.aiplatform.base:PipelineJob deleted. . Resource name: projects/931647533046/locations/us-central1/pipelineJobs/custom-training-pipeline-20220809133409


In [27]:
# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
exp.delete()

Deleting Context : projects/931647533046/locations/us-central1/metadataStores/default/contexts/vertex-ai-dev-classification-xgboost-exxfumy6


INFO:google.cloud.aiplatform.base:Deleting Context : projects/931647533046/locations/us-central1/metadataStores/default/contexts/vertex-ai-dev-classification-xgboost-exxfumy6


Delete Context  backing LRO: projects/931647533046/locations/us-central1/metadataStores/default/contexts/vertex-ai-dev-classification-xgboost-exxfumy6/operations/971247342923546624


INFO:google.cloud.aiplatform.base:Delete Context  backing LRO: projects/931647533046/locations/us-central1/metadataStores/default/contexts/vertex-ai-dev-classification-xgboost-exxfumy6/operations/971247342923546624


Context deleted. . Resource name: projects/931647533046/locations/us-central1/metadataStores/default/contexts/vertex-ai-dev-classification-xgboost-exxfumy6


INFO:google.cloud.aiplatform.base:Context deleted. . Resource name: projects/931647533046/locations/us-central1/metadataStores/default/contexts/vertex-ai-dev-classification-xgboost-exxfumy6


In [28]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}